#  Evaluate Detection Model with FiftyOne

This walkthrough demonstrates how use FiftyOne to perform hands-on evaluation of your detection model.

It covers the following concepts:
* Loading a dataset with detections
* Adding detection predictions
* Sample-wise MSCOCO evaluation
* Sorting and searching samples by model performance
* Visualizing true-positives and false-positives
* Querying your dataset for a custom insight

# Setup

Install `torch` and `torchvision`, if necessary:

In [ ]:
# Modify as necessary (e.g., GPU install). See https://pytorch.org for options
!pip install torch
!pip install torchvision

Import the FiftyOne zoo and download the MSCOCO validation split to `~/fiftyone/coco-2017/validation`

In [ ]:
import fiftyone.zoo as foz

In [ ]:
dataset = foz.load_zoo_dataset("coco-2017", "validation")

Initialize Faster-RCNN and download pretrained weights:

In [ ]:
# Run the model on gpu if it is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.to(device)
model.eval()

# Generate Predictions
Run Faster-RCNN on every sample in the validation dataset and add detections to our FiftyOne dataset.
Predictions are added to each sample in a new field we will call `faster_rcnn`

In [3]:
# ETA is installed with FiftyOne
# etai provides functionality to read images into memory
import eta.core.image as etai
from torchvision.transforms import functional as F


# Add predictions
for sample in dataset:
    image = etai.read(sample.filepath)
    image = F.to_tensor(image).to(device)
    
    preds = model([image])[0]
    
    labels = preds["labels"].cpu().detach().numpy()
    scores = preds["scores"].cpu().detach().numpy()
    boxes = preds["boxes"].cpu().detach().numpy()
    
    detections = []
    for label, score, box in zip(labels, scores, boxes):
        # Compute relative bounding box coordinates
        x1, y1, x2, y2 = box
        rel_box = [x1/w, y1/h, (x2-x1)/w, (y2-y1)/h]
        
        detections.append(fo.Detection(
            label=label,
            bounding_box=rel_box,
            confidence=score
        ))

    sample["faster_rcnn"] = fo.Detections(
        detections=detections
    )
    sample.save()

print("Finished adding predictions")

Finished adding predictions


# Evaluate Detections
Use MSCOCO detection evaluation provided within FiftyOne to threshold detections and compute AP for each sample

Threshold all detections to remove any detections lower than 0.5 confidence

In [ ]:
for sample in dataset:
    sample["faster_rcnn_5"] = sample["faster_rcnn"].filter(0.5)
    sample.save()

Match detections to ground truth and compute AP according to MSCOCO evaluation

In [ ]:
dataset.evaluate("faster_rcnn_5")

Evaluation automatically added 
* `AP`- mAP over IoU values of [0.5,0.55,0.6,...,0.95]
* `AP_0_5` - mAP over IoU 0.5
* `AP_0_75` - mAP over IoU 0.75
* `AP_small` - mAP over objects with small bounding boxes
* `AP_medium` - mAP over objects with medium bounding boxes
* `AP_large` - mAP over objects with large bounding boxes 

Implementations can be found in the `pycocotools` repository: https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocotools/cocoeval.py

In [ ]:
print(dataset)

Additionally, every detection stores which ground truth boxes they were matched with, if any.

In [ ]:
sample = next(dataset)

print(sample["faster_rcnn_75"].detections[0])

Compute true and false positives for each sample 

In [ ]:
for sample in dataset:
    tp_dets = []
    fp_dets = []
    matched_gt = []
    missed_gt = []
                                                                                                                                                                                                            
    gt_ids = {x.attributes["coco_id"].value: x for x in sample.ground_truth.detections}
    
    for det in sample['faster-rcnn_75'].detections:
        gtid75 = det.attributes['gtId_75'].value
        if gtid75 == 0:
            fp_dets.append(det)
        else:
            tp_dets.append(det)
            matched_gt.append(gt_ids[gtid75])
            
    missed_gt = [x for x in gt_ids.values() if x not in matched_gt]
    sample['faster_rcnn_75_tp'] = fol.Detections(detections=tp_dets)
    sample['faster_rcnn_75_fp'] = fol.Detections(detections=fp_dets)
    sample['faster_rcnn_75_gt_matched'] = fol.Detections(detections=matched_gt)
    sample['faster_rcnn_75_gt_missed'] = fol.Detections(detections=missed_gt)
    sample.save()

# Visualize Detections
Launch the FiftyOne app and easily view ground truth and predicted bounding boxes.

In [5]:
session = fo.launch_app(dataset=dataset)

App launched


![launch](images/eval_dets/launch_app.png)

All fields are shown as togglable bubbles on the left sidebar which can be used to switch between viewing true positives, false positives, missed ground truth boxes, etc.

![bubbles](images/eval_dets/togge_bubbles.png)

## Dataset Views
A `DatasetView` can also be used to search, sort, or splice your dataset for you to look at different views of the samples. 

Individual samples can be selected and a `DatasetView` can be created to look at just those samples.

In [6]:
selected_samples = session.selected
session.view = dataset.view().select(selected_samples)

![selected](images/eval_dets/selected.png)

Reset the session dataset to show the entire dataset again.

In [11]:
session.dataset = dataset

`AP` was calculated for each sample during evaluation, we can make a `DatasetView` that sorts by `AP` to look at the best and worst predictions that the model had.

In [9]:
session.view = dataset.view().sort_by("AP", reverse=True)

![ap_rev](images/eval_dets/ap_rev.png)

In [10]:
session.view = dataset.view().sort_by("AP")

![ap](images/eval_dets/ap.png)

The evaluation also stored `AP` by bounding box size in the fields named `AP_small`, `AP_medium`, and `AP_large`. This can be used to look at how the model performed on small samples, for example.

In [24]:
session.view = dataset.view().sort_by("AP_small", reverse=True)

![ap_small](images/eval_dets/ap_small.png)

In [25]:
session.view = dataset.view().sort_by("AP_small")

![ap_small_rev](images/eval_dets/ap_small_rev.png)

As we can see, some samples did not contain any small objects. We can define a query using [MongoDB queries]( https://docs.mongodb.com/manual/tutorial/query-documents/) and the `.match()` method of a `DatasetView` in order to find all samples that have an `AP_small` of `0` or `None`. We can then remove them from our view and only look at samples that contain small objects.

In [18]:
# Match all samples that where the field `AP_small` equals 0 or None
query = {"$or" : [{"AP_small" : 0}, {"AP_small" : None}]}
no_small_objs_view = dataset.view().match(query)

# Get all ID's of the samples that have no small objects
# Then create a new view without those samples
no_small_ids = [s.id for s in no_small_objs_view]
small_view = dataset.view().exclude(no_small_ids)

# Visualize this new view that contains only samples with small objects
session.view = small_view.sort_by("AP_small")

![ap_small_only](images/eval_dets/ap_small_only.png)